In [12]:
import sqlite3
import pandas as pd
from IPython.display import display, Markdown

# Connect to SQLite database
conn    = sqlite3.connect('adzuna_jobs.db')

# Read a table into a dataframe
jobs_df = pd.read_sql_query("SELECT * FROM jobs", conn)

# close the connection
conn.close()

display(Markdown('### Jobs Dataframe'))
display(jobs_df.tail())
display(Markdown('---'))
display(Markdown('Descriptive Statistics'))
display(jobs_df.describe())
display(Markdown('---'))
display(Markdown('Missing Values'))
display(jobs_df.isnull().sum())
display(Markdown('---'))
display(Markdown('Data Types'))
display(jobs_df.dtypes)
display(Markdown('---'))


### Jobs Dataframe

,id,title,company,location,category,salary_max,salary_min,latitude,longitude,contract_type,contract_time,description,created,url
145,5288670448,Data Analyst,Kahana & Feld Llp,"Huntington, Orange County",IT Jobs,74880.0,54080.0,33.675315,-117.817344,None,full_time,The Data Analyst will support the Aderant soft...,2025-07-06T03:37:07Z,None
146,5274701695,Data Analyst,Anine Bing,"Los Angeles, Los Angeles County",IT Jobs,110000.0,90000.0,NaN,NaN,None,full_time,ANINE BING is looking for a strategic and hand...,2025-06-29T06:43:32Z,None
147,5306322810,Data Analyst,Apex Systems,"Pomona, Los Angeles County",IT Jobs,83200.0,52000.0,33.994555,-117.813558,None,None,Job: 2081463 Job Description: Data Analyst Rem...,2025-07-16T07:28:22Z,None
148,5310800188,Data Analyst,Cypher Analytics,"San Diego, San Diego County",Engineering Jobs,100000.0,80000.0,32.726967,-117.164709,None,full_time,Full-time Description About Our Company: At Cy...,2025-07-18T18:24:15Z,None
149,5248155725,Data Analyst,Lbs Financial Credit Union,"Westminster, Orange County",IT Jobs,86091.0,68868.0,33.749964,-117.989162,None,full_time,Are you a detail-oriented professional who exc...,2025-06-13T05:38:22Z,None


---

Descriptive Statistics

,salary_max,salary_min,latitude,longitude
count,150.000000,150.000000,77.000000,77.000000
mean,80226.384000,73926.877333,38.212112,-97.439284
std,31667.131773,28166.485779,4.327679,31.570481
min,27083.250000,25000.000000,32.726967,-122.466538
25%,54882.587500,51576.497500,33.954720,-120.943179
50%,75175.000000,69131.685000,37.928543,-117.393600
75%,99658.497500,93480.715000,40.750133,-73.989780
max,175000.000000,156000.000000,51.515704,0.035485


...

id                 0
title              0
company            0
location           0
category           0
salary_max         0
salary_min         0
latitude          73
longitude         73
contract_type    118
contract_time     91
description        0
created            0
url              150
dtype: int64

---

Data Types

id                object
title             object
company           object
location          object
category          object
salary_max       float64
salary_min       float64
latitude         float64
longitude        float64
contract_type     object
contract_time     object
description       object
created           object
url               object
dtype: object

---